In [14]:
from urllib.request import urlopen
from urllib.error import HTTPError
from urllib.parse import urljoin
from bs4 import BeautifulSoup
import random
import time

In [15]:
baseUrl = 'https://www.imdb.com/'

classes = [['Titulo', 'ipc-metadata-list-summary-item__t'], 
           ['Data', 'ipc-title__text'], 
           ['Genero', 'ipc-metadata-list-summary-item__li'], 
           ['Link', 'ipc-metadata-list-summary-item__t']]
pages = set()
info = []

In [16]:
from urllib.request import Request


def getLinks(pageUrl):
    global pages, baseUrl
    fullUrl = urljoin(baseUrl, pageUrl)
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'}

    try:
        request = Request(fullUrl, headers=headers)
        html = urlopen(request)
        bs = BeautifulSoup(html, 'html.parser')
        time.sleep(random.randint(1, 3)) 

        for link in bs.find_all('a'):
            href = link.get('href')
            newPage = urljoin(baseUrl, href)
            if newPage not in pages:
                if 'title' in href:
                    pages.add(newPage)
                    print(newPage)
                    getLinks(href)
    except HTTPError as e:
        print(f'Erro HTTP [{e.code}] tentando acessar [{fullUrl}]')
    except Exception as e:
        print(f'Deu pau :( [{e}]')

In [17]:
getLinks('/calendar/')

https://www.imdb.com/search/title/?title_type=podcastSeries&ref_=nv_pod
https://www.imdb.com/title/tt12326830/?ref_=sr_i_1
https://www.imdb.com/title/tt12326830/episodes/?ref_=tt_ov_epl
https://www.imdb.com/title/tt12326830/?ref_=ttep_ov


KeyboardInterrupt: 

In [ ]:
for page in pages:
    print(page)